In [ ]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.r = rnd((1,1))
        self.R = self.r.T@self.r
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['R','Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.R = self.r.T@self.r
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ ii(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
        
        

<h3>We are conintuing from the last time to prepare inputs into the minimize function, listed below.</h3>

In [ ]:
from scipy.optimize import minimize

minimize(
    fun = # The function that we would like to minimize;
    x0 = # Starting values of the parameters;
    method = # The numerical method to be used for minimization;
    constraints = # Define the valid sets of the values of the parameters;
    tol = # Tolerance for convergense;
    options = # Options for the minimization process; e.g., the maximum number of 
              # iterations to be attempted.
)

In [ ]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.r = rnd((1,1))
        self.R = self.r.T@self.r
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['R','Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.R = self.r.T@self.r
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ ii(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
    def fun2min(self,column): # This is the function we will be minimizing.
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        self.run() # Run the Kalman filter with the new parameters.
        return -self.ll # Return the negative of the lig likelihood.
    

AR(1) model: $ y_t = \delta + \phi y_{t-1} + \epsilon_t.$

Here, if $\phi = 1$, then we have an integrated process (random walk), which is non-stationary.

If, $\phi > 1$ or $\phi < -1$, then we have an explosive process.

So, for a well-behaved AR(1) model, we need $-1<\phi<1$.

For the process \{y_t\} to be stationary, its long-term mean and variance shound be finite (i.e., not infinite).  Let's calculate the long-term mean and variance:

$\mathrm{E}(y) = \delta + \phi \mathrm{E}(y) \Longrightarrow\mathrm{E}(y) = \frac{\delta}{(1-\phi)}.$<br><br>
$\mathrm{var}(y) = \phi^2 \mathrm{var}(y) + \mathrm{var}(\epsilon) \Longrightarrow \mathrm{var}(y) = \frac{\mathrm{var}(\epsilon)}{(1-\phi^2)}$.

In the case of a VAR(1) (i.e., <i>vector</i> autoregresive model), similar conditions are available, and they are on the slides.

In [18]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math
from scipy.optimize import minimize

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.r = rnd((1,1))
        self.R = self.r.T@self.r
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['R','Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.R = self.r.T@self.r
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ inv(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
    def fun2min(self,column): # This is the function we will be minimizing.
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        self.run() # Run the Kalman filter with the new parameters.
        print (self.ll) # Report progress.
        return -self.ll # Return the negative of the lig likelihood.
    def F_constraint(self,column):
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        F = self.pars.F
        I = np.eye(F.shape[0])
        try: 
            an_inverse = inv(I-F)
            return 1
        except: return -1
    def estimate(self):
        starting_column = self.pars.pack()
        constr = {"type": "ineq", "fun": self.F_constraint}
        opt = {"maxiter": 1000}
        solution = minimize(
            fun = self.fun2min,
            x0 = starting_column,
            method = "COBYLA",
            constraints = constr,
            tol = 0.01,
            options = opt)
        self.optimum = solution
        return solution
        
        

In [19]:
ibm = the_model('IBM.csv')

In [20]:
ibm.estimate()

-160.18698903711237
-296.78775544304864
-170.4280945127897
-165.97682330112525
-296.0946240214732
-311.55576592994737
-177.38243323182763
-216.9759976393064
-171.3339404157653
-246.57760632718518
-160.25728212682648
-160.1996862122153
-160.19367495243614
-171.73377406520655
-165.97314310169816
-162.87171724998333
-160.25583348959069
-160.2573556156333
-160.2700687861969
-160.20032023522631
-160.2007796296913
-160.204624948821
-160.1951546500962
-160.1954469198054
-160.19796012206513
-164.51724020758908
-162.33130532303915
-163.62713588103708
-161.06217270316552
-161.67362073041474
-161.41116234301492
-161.86062266225392
-160.69913210652985
-160.98350167376842
-558.5579237020198
158.52437317562058
-316.65241041991266
397.5848767682497
293.7287881267583
-104.86031199997682
-140.87191170944766
475.1767232690929
429.5911814351969
40.2268676946375
411.348734303563
139.94308745280418
448.1296469400909
146.58668777158593
455.0354787964428
149.86721408547774
470.15254903757796
149.880232469625

     fun: -531.9517500350483
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 356
  status: 1
 success: True
       x: array([[-0.27795684],
       [ 0.56907869],
       [ 0.47265849],
       [-0.40989398],
       [ 0.38743525],
       [ 0.04397149],
       [-0.00925719],
       [-0.53764016],
       [-0.32695462],
       [ 0.4430928 ],
       [ 0.1347814 ],
       [ 0.29188618],
       [ 0.58787989],
       [-0.3054487 ],
       [ 0.03041564],
       [ 0.2933157 ],
       [ 0.26130601],
       [ 0.85327478],
       [ 0.22081186],
       [ 0.18069736],
       [ 0.98883586],
       [ 0.2997681 ],
       [ 0.07067764],
       [ 0.2401056 ],
       [ 0.56546453],
       [-0.23226716],
       [ 0.08713597],
       [ 0.45328697],
       [ 0.860037  ],
       [ 0.41386205],
       [ 0.77326929],
       [ 0.22354732],
       [ 0.15624205],
       [ 0.05074207]])

In [21]:
ibm.optimum

     fun: -531.9517500350483
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 356
  status: 1
 success: True
       x: array([[-0.27795684],
       [ 0.56907869],
       [ 0.47265849],
       [-0.40989398],
       [ 0.38743525],
       [ 0.04397149],
       [-0.00925719],
       [-0.53764016],
       [-0.32695462],
       [ 0.4430928 ],
       [ 0.1347814 ],
       [ 0.29188618],
       [ 0.58787989],
       [-0.3054487 ],
       [ 0.03041564],
       [ 0.2933157 ],
       [ 0.26130601],
       [ 0.85327478],
       [ 0.22081186],
       [ 0.18069736],
       [ 0.98883586],
       [ 0.2997681 ],
       [ 0.07067764],
       [ 0.2401056 ],
       [ 0.56546453],
       [-0.23226716],
       [ 0.08713597],
       [ 0.45328697],
       [ 0.860037  ],
       [ 0.41386205],
       [ 0.77326929],
       [ 0.22354732],
       [ 0.15624205],
       [ 0.05074207]])

In [25]:
ibm.optimum.x

array([[-0.27795684],
       [ 0.56907869],
       [ 0.47265849],
       [-0.40989398],
       [ 0.38743525],
       [ 0.04397149],
       [-0.00925719],
       [-0.53764016],
       [-0.32695462],
       [ 0.4430928 ],
       [ 0.1347814 ],
       [ 0.29188618],
       [ 0.58787989],
       [-0.3054487 ],
       [ 0.03041564],
       [ 0.2933157 ],
       [ 0.26130601],
       [ 0.85327478],
       [ 0.22081186],
       [ 0.18069736],
       [ 0.98883586],
       [ 0.2997681 ],
       [ 0.07067764],
       [ 0.2401056 ],
       [ 0.56546453],
       [-0.23226716],
       [ 0.08713597],
       [ 0.45328697],
       [ 0.860037  ],
       [ 0.41386205],
       [ 0.77326929],
       [ 0.22354732],
       [ 0.15624205],
       [ 0.05074207]])

In [26]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math
from scipy.optimize import minimize

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.r = rnd((1,1))
        self.R = self.r.T@self.r
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['R','Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.R = self.r.T@self.r
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ inv(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
    def fun2min(self,column): # This is the function we will be minimizing.
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        self.run() # Run the Kalman filter with the new parameters.
        print (self.ll) # Report progress.
        return -self.ll # Return the negative of the lig likelihood.
    def F_constraint(self,column):
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        F = self.pars.F
        I = np.eye(F.shape[0])
        try: 
            an_inverse = inv(I-F)
            return 1
        except: return -1
    def estimate(self):
        starting_column = self.pars.pack()
        constr = {"type": "ineq", "fun": self.F_constraint}
        opt = {"maxiter": 1000}
        solution = minimize(
            fun = self.fun2min,
            x0 = starting_column,
            method = "COBYLA",
            constraints = constr,
            tol = 0.01,
            options = opt)
        self.optimum = solution
#        best_parameters = solution.x
#        self.pars.unpack(best_parameters)
        self.pars.unpack(solution.x)
        self.run()
        
        

In [27]:
ibm = the_model('IBM.csv')
ibm.estimate()

-116.39108212735395
-221.62831016220792
-196.06269285728408
-172.92086967444368
-187.09857896181254
-190.92774965870834
-167.78275181858825
-180.47916057969636
-167.67270489096822
-188.29171446541451
-116.63662807441631
-116.60302817471681
-116.56717923491135
-120.88337230610134
-128.36772604682054
-124.84026478656934
-116.51857073909301
-116.51355500517093
-116.4495778523843
-116.4841894253666
-116.47349764709219
-116.42740530497197
-116.47684655835458
-116.47388578104022
-116.42683496632061
-122.60501991933553
-120.39110331723653
-122.58150401241086
-117.18657661816462
-120.7186056849405
-118.92731496509808
-120.66495627999724
-119.11764203659746
-118.14630327430672
-708.7225358931946
-363.96745757669504
47.64719106638823
-277.9161723663423
144.3778697259718
-65.57811597941723
221.11658624250128
56.77749846593913
278.29212608080337
142.82162423198352
310.6672601386061
230.38033221855397
211.79213421536724
174.28540460764842
222.78911998116772
362.7851911457926
163.05208322710604
343.

453.77993351959867
454.09892620237576
454.01438457567275
454.0566673961513
454.1077404672964
454.0850889687522
453.9787721535685
454.12168182352667
454.0968132069774
454.11277942617136
454.1079097442819
454.1259937467967
454.09419966831115
454.1312156468892
454.1263039387224
454.16078037382607
454.1785271751213
454.17528645730783
453.7117117505059
453.7667842026477
454.2939012585756
454.3236783148804
454.68263640006654
454.90926414318835
454.879891093207
455.00457005667806
455.21013450386016
455.3703071865548
454.92113749507183
455.40790352707927
455.4095937180676
455.26822175494635
455.4100921637403
455.16162342352845
455.3853375445749
455.3481634575793
454.8537624179048
455.55291091593756
455.41871546591864
455.5609995221714
455.56413196327804
455.51582385523074
455.5604426427959
455.5368413574557
455.5511441555214
455.4974048782074
455.56418189469576
455.5368039948721
455.5628182830465
455.52845964750634
455.5622001269761
452.82030936959916
455.58606584551956
455.5840473554557
455.5

In [28]:
ibm.optimum

     fun: -456.031096930117
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 531
  status: 1
 success: True
       x: array([[ 0.10066769],
       [ 0.36694397],
       [ 0.54640771],
       [ 0.38046654],
       [-0.31288146],
       [ 0.53230447],
       [ 0.41351315],
       [ 0.33674524],
       [-0.15964913],
       [ 1.0472501 ],
       [ 0.10490576],
       [ 0.21052643],
       [ 0.75982019],
       [ 0.13031851],
       [ 0.19444467],
       [ 0.87230189],
       [ 0.84275867],
       [ 0.12256387],
       [ 1.09331839],
       [ 0.3996064 ],
       [ 0.2017029 ],
       [ 0.36659467],
       [ 0.75268427],
       [ 0.33679708],
       [ 1.11736403],
       [ 0.56900247],
       [ 0.90207596],
       [ 0.43819722],
       [ 0.58432995],
       [ 0.15614446],
       [ 1.10652892],
       [ 0.74683643],
       [-0.06041798],
       [-0.04983671]])

In [29]:
for t in ibm.sample: print(t.upd.beta.T)

[[0.45936723 0.30780944 0.2716591 ]]
[[2.51957876 0.71687675 1.35634104]]
[[-0.3438835   0.10491369 -0.24425984]]
[[ 0.38686323 -0.21099395 -0.05656116]]
[[0.60185664 0.27620929 0.22743332]]
[[-0.18558558 -0.19304885 -0.26779662]]
[[1.39572003 0.29700917 0.35774474]]
[[0.39372454 0.26564339 0.39692829]]
[[ 0.28071873  0.00328643 -0.04151435]]
[[2.00568674 0.5623235  0.57914171]]
[[2.00709428 1.18704633 1.91113677]]
[[2.0049245  1.05663327 1.47050365]]
[[0.75807246 0.35780186 0.55396734]]
[[ 0.51404554  0.08245778 -0.02951602]]
[[0.92162367 0.3086982  0.55376613]]
[[1.50085476 0.70672023 0.99509635]]
[[0.4696912  0.15102022 0.1933864 ]]
[[1.63946387 0.62153101 0.92824012]]
[[1.57394273 0.81354615 1.22071941]]
[[1.12635923 0.50066018 0.64546125]]
[[1.30624811 0.59281427 0.95517818]]
[[1.54371732 0.76124366 1.09431845]]
[[0.99493479 0.41727077 0.57259916]]
[[0.44650598 0.12268639 0.1480512 ]]
[[0.85825295 0.21839054 0.29307343]]
[[0.88109688 0.43151782 0.6863528 ]]
[[1.29658173 0.55041775

In [30]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math
from scipy.optimize import minimize

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.r = rnd((1,1))
        self.R = self.r.T@self.r
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['R','Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.R = self.r.T@self.r
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ inv(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
    def fun2min(self,column): # This is the function we will be minimizing.
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        self.run() # Run the Kalman filter with the new parameters.
        print (self.ll) # Report progress.
        return -self.ll # Return the negative of the lig likelihood.
    def F_constraint(self,column):
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        F = self.pars.F
        I = np.eye(F.shape[0])
        try: 
            an_inverse = inv(I-F)
            return 1
        except: return -1
    def estimate(self,tol=0.01, maxit = 1000):
        starting_column = self.pars.pack()
        constr = {"type": "ineq", "fun": self.F_constraint}
        opt = {"maxiter": maxit}
        solution = minimize(
            fun = self.fun2min,
            x0 = starting_column,
            method = "COBYLA",
            constraints = constr,
            options = opt)
        self.optimum = solution
        self.pars.unpack(solution.x)
        self.run()
        
        

In [31]:
ibm = the_model('IBM.csv')
ibm.estimate()

-521.2828425804001
-576.4135465982614
-563.5371669776188
-566.4625550789603
-581.2040519128958
-650.9448896013026
-610.4519076588455
-569.0840504564284
-594.5721499985867
-635.0954151633456
-521.4597888207048
-521.5906276060543
-521.602445432074
-521.7185541326699
-522.633997021636
-522.3488830428936
-521.3089675669523
-521.3414388948471
-521.3317502138692
-521.3331670061092
-521.3854533831172
-521.3700865815376
-521.3366374905851
-521.3910803966209
-521.3751187758538
-522.5757128417
-522.2872527876164
-522.1486428291893
-521.2732112488225
-521.307268990363
-521.3774119972288
-521.4711063191046
-521.7307129652578
-521.7368750385161
-825.0424707593746
394.84270722003953
458.5947690057258
137.27664225433193
-226.60267015444805
351.46487580521944
149.38570684284943
383.88837334329526
451.66299504790084
460.91397203207936
454.05693767746794
445.9243101250716
433.42873997750246
444.7803460973165
7.7446631403501325
447.10008070157005
455.2501577136257
432.0667474035235
458.1711783981735
446.

515.7645343095433
515.769945758179
515.7517348946711
515.7602914989193
515.6966736019467
515.7408292507741
515.7338680880192
515.7447310942634
515.771546009255
515.7676854683644
515.7434307040608
515.7774474420885
515.7561493752589
515.7780692475327
515.7871786505449
515.7899145459942
515.7444846583957
515.7834535483324
515.8027043753834
515.8030269329909
515.7955020418265
515.8030748240444
515.775753611708
515.8026563422623
515.7647879239927
515.7849709629106
515.6797602740729
515.8059862391094
515.7636589304103
515.8091553760129
515.8123666709556
515.8139574083017
515.7850417529169
515.8171813153697
515.809337629734
515.8211167933354
515.7640961481713
515.8203942642326
515.7713295851336
515.8235566489792
515.7891743276301
515.8234047380447
515.7964636153441
515.8236627065008
515.8019643526359
515.8236591623739
515.8147465299645
515.8236090725507
515.8094792749042
515.8249789821023
515.7759667684029
515.8248788302361
515.8073237327922
515.8475809165542
515.8264139738995
515.8417189322

518.3754520179216
518.436009899013
518.4480597706174
518.4455965306772
518.4467496072559
518.4495977725808
518.3896441850975
518.4478984671916
518.4902595935943
518.5115438919587
518.5075795456667
518.4870502765204
518.5116348385937
518.5122099452507
518.5125588545565
518.5086355732107
518.51284981725
518.4974827546645
518.5236600104644
518.555869400469
518.5559671386023
518.5074070619539
518.5583896024987
518.6195176095725
518.5418664723368
518.6188981531063
518.5410495415765
518.6176306936807
518.6422275790459
518.6348443930244
518.5611731761046
518.633917059412
518.571397382955
518.6415791493473
518.6367221130196
518.6130762367483
518.6497502289074
518.6394163722684
518.6584054037493
518.6521753431776
518.6550095899684
518.6527917559871
518.6941180283668
518.6778636573711
518.6947467353837
518.6969846423316
518.7122226608045
518.7156921154339
518.7168420034574
518.7168342923666
518.7147267043049
518.71786712685
518.7175188166631
518.7173650196088
518.6492259597912
518.717848158396
5

In [32]:
for t in ibm.sample: print(t.upd.beta.T)

[[ 0.84301587  0.50301219 -0.30707024]]
[[ 1.01888088 -0.64108991  0.25337557]]
[[ 0.46234539 -0.10400966 -0.58535847]]
[[ 0.4456626  -0.5815397  -0.31343283]]
[[ 0.81531731  0.04303739 -0.24542112]]
[[ 0.19334399 -0.76746164 -0.50270901]]
[[ 1.10322818  0.25001596 -0.04519828]]
[[ 0.92236208 -0.05241307 -0.15663346]]
[[ 0.52432102 -0.36018881 -0.39108756]]
[[1.45524721 0.6966953  0.05212879]]
[[1.5055863  0.49315413 0.11410879]]
[[ 1.22616295  0.22212061 -0.04322066]]
[[ 1.12230292  0.1444926  -0.08220342]]
[[ 1.00548842  0.03174552 -0.13069338]]
[[ 1.00244135  0.05279817 -0.13003322]]
[[ 1.15502111  0.1945846  -0.04819374]]
[[ 0.78110087 -0.20412213 -0.23445241]]
[[ 1.2915859   0.4163774  -0.00207715]]
[[ 1.07064332  0.01697707 -0.07054963]]
[[ 1.04450069  0.1613305  -0.14767263]]
[[ 1.02666769  0.02836904 -0.09742655]]
[[ 1.15773356  0.22123046 -0.06134341]]
[[ 1.08103426  0.09740439 -0.08764508]]
[[ 0.76865351 -0.18324842 -0.25144718]]
[[ 1.14070951  0.29072008 -0.07675993]]
[[ 0.9

In [33]:
ibm.estimate(tol = 0.001, maxit = 5000)

518.830170059156
273.32379705517457
487.0424460574602
451.1520451314392
471.7798792454331
512.4495333275464
461.81067952693945
503.5833014274926
512.3768014668229
461.9201301548995
518.6135087458063
518.8002171130062
518.5823973774807
411.52555677998424
484.61026840236764
464.0241956691974
518.7692486782881
518.7249973840543
518.7384132000491
518.7847624071998
518.8290273755003
518.8143037327322
518.7499840578832
518.7009140868118
518.7158797915303
494.7667623814578
491.91989100124005
483.9163903148234
510.47165836195455
522.4935588757694
522.7046655120375
515.6145573899989
518.7789377002217
514.4414234223101
-351.81514698540786
-372.62852024341373
521.5568484272472
81.58588250824283
473.03447518975304
516.1877212442507
495.40252797028216
526.7582301117924
489.2701099448756
527.4292548280688
504.24050914926534
527.4306001871931
522.1118291374321
525.5364417283515
516.1715707302059
528.1211406394533
516.0709749027039
528.113986766749
522.5559753005678
528.1211292373251
521.3824403315367

559.227981376568
320.3722898877647
559.3226668845218
513.1720111699102
559.3315461571541
303.540184405857
559.3744207346891
514.9931110392756
559.37244823804
306.60397178965997
559.3241829667172
306.6257950524301
559.3395798191874
306.6728818341057
559.3966015975831
304.3647573894107
559.3942730159097
515.1936655805428
559.3818936170286
304.49999895537917
559.3902759394556
304.5125799217704
559.4277739776812
305.1322651255575
559.4860820739174
514.8246716954783
559.5270500687465
301.755266223799
559.7348453872605
514.0523690486682
559.7199058224196
320.88638569770046
559.7280904635686
320.8980797796311
559.7723685820525
328.2634483791118
559.7735756549935
513.2888160788582
559.7867860619731
321.97765924970963
559.7793178752712
513.9396162364727
559.7683556925612
322.08995549337016
559.835647041431
328.29801654252225
559.649830709211
513.2387464078066
559.6556742473068
330.8997857724548
559.8477329972903
337.10698678519776
559.7718359184482
512.7289303226944
559.9023833246569
320.941608

563.5519281528371
548.3250842120607
563.5240877756086
527.7946449027246
563.5606274631775
527.3586002912082
563.5736055049232
548.31579961197
563.5726827143396
527.8629316619787
563.5736717416015
527.9729917298864
563.5659856362764
548.2790028159137
563.6407319073934
529.6407264927113
563.633584010379
547.8303396506361
563.6443924383782
528.7815833591329
563.6662513301351
547.9607240220477
563.6370944533364
529.2786293529509
563.6754487952734
528.7004042879171
563.6740925887373
548.1561590909226
563.6850873802264
528.4500407005543
563.6984907922372
548.2400585431494
563.7011614852777
527.7372588403825
563.7432511614736
548.7406417164245
563.765670953711
527.0777021363538
563.768994172007
548.2619177818987
563.7695396978398
527.4701530996853
563.7989150640514
548.1390929471097
563.7990443162222
529.0184755735487
563.8364868612828
547.6677984079957
563.8425828135777
530.9846496477671
563.8350154193168
547.6550567427817
563.8384152973354
531.0103341399696
563.7547936371324
530.89740287040

530.1119889376145
565.4767007423293
530.1122770265183
565.476343756217
530.1064958673284
565.4669422082882
530.1114365959827
565.4779452999666
529.6605826031283
565.4811567078058
549.73808561477
565.472249969727
530.122028355858
565.4843248127497
530.7952489752269
565.5231843305938
549.126127101166
565.5212253555321
532.2689769724325
565.5288279940586
531.7255809062144
565.5380553587571
549.1611420863981
565.5448136571472
531.9397537083842
565.5598786753253
549.4767780524397
565.5695880666378
531.1879943510544
565.5794639353114
549.4847453808453
565.5946392052108
530.6532457445983
565.5708754458285
549.7088632573511
565.5849885919678
530.6452085784362
565.5769144273212
530.664404289855
558.6265805813224
565.6191814130484
560.9907394018037
565.6172358836243
558.6373990660263
565.631440385845
558.7976208531752
565.6321232144594
560.7923573391216
565.6285570915695
559.0245986475
565.6310235807058
559.0255737732406
565.6281264279952
559.0310437617848
565.623575741232
559.0510016332279
565.

566.3335919792469
561.6764785001313
566.3348895879237
559.3014014337368
566.3288838720111
561.7129798046399
566.3355547881813
558.9080047029183
566.3384939052742
561.8644551217767
566.3411820733942
559.11696003251
566.3426245323666
561.7823894074738
566.3439952497744
559.3300440861036
566.3342616296189
561.7083943980572
566.365036780078
559.4054546096805
566.3683453580943
561.7436794747613
566.3785384799743
559.4634900240477
566.380250532171
561.7134030641215
566.3817410523955
559.3403732784292
566.3863576071756
561.7710383165012
566.3911049752079
559.3942270366181
566.3942997702084
561.8146097737052
566.3931382919111
559.2699939522681
566.4002085256699
559.4896715541141
566.4020544239789
561.6619349885382
566.3965540639439
559.5444472214648
566.401316336909
559.5438594603711
566.4226988414348
559.3181537781145
566.4271137848075
561.6911926875687
566.4283828103668
560.1595832133735
566.4294867524992
561.1963726767658
566.4235275384361
560.4163464332624
566.4130911489164
560.41164845821

...

567.4213925754265
565.9965300662122
567.4224604550001
566.0538479448024
567.4208985423993
566.0016328759117
567.4238767384591
566.0269466804345
567.4272074701908
565.990945485247
567.4281098993783
566.0602313728698
567.4260449952019
565.9883478077982
567.4300858366637
565.8931088922876
567.425798123204
566.1366399609354
567.428033698874
565.8911724205324
567.4304965784947
565.9162413791712
567.428015330378
566.1194653506827
567.4323075553799
565.9006084729059
567.4324823109264
566.1699084605781
567.4402020435847
565.790115553076
567.4330904421316
566.2294908740284
567.4390973403478
565.7948006510337
567.441188925711
565.7912384366723
567.442186499187
566.2489032767105
567.4461572370528
565.7940552360803
567.4467665376271
566.2468916126259
567.4491332022869
565.7989733778377
567.449707336436
566.265573834489
567.4525459256635
565.7911763845137
567.4446559496181
566.2468764767585
567.4520391894507
565.7964782078385
567.4561022949397
565.8187841834236
567.4569347011591
566.2463118508705
5

566.1983895182899
567.7528324037862
566.2035963118605
567.7575336656042
566.3998119002282
567.7576871601941
566.2856958267432
567.7577561809456
566.4289874133704
567.7570150236389
566.249773389599
567.7590002751142
566.2050296183575
567.758575273418
566.4641261593557
567.7598810554454
566.082291641244
567.7544397618224
566.5571581672313
567.7549530349294
566.0874978236861
567.7577142508233
566.0882183348789
567.759907623338
566.0826920171216
567.7609818071779
566.5235346817884
567.7621836796027
566.1524128502884
567.763308978936
566.5257576331744
567.7613795213638
566.1351236470678
567.7678267932484
566.0282662789496
567.7678705860297
566.6257053166017
567.7696366949423
566.0187278865492
567.7713395016939
566.6397762261124
567.7598940923216
566.0158626125304
567.7723876651653
566.0182871051597
567.7722748523178
566.6367391483245
567.7744558396305
566.020417252294
567.7772134295724
566.6331789504765
567.776403742889
566.0340274358466
567.7786104679365
566.0144172503957
567.77958838125
5

568.0480349873641
566.7372552064625
568.0485095855272
566.4824519940117
568.0486223759304
566.7101662498412
568.056272098918
566.5260858738712
568.0551335025683
566.7196489451985
568.0610614163386
566.561189117015
568.0628765773753
566.7244551727205
568.0637736655477
566.4622130185143
568.0615715005022
566.7806544207763
568.0606355721745
566.4592590395006
568.0614033734464
566.4601735980315
568.0621328912458
566.4607164007165
568.0639316702974
566.4519804499813
568.0636423071658
566.7873197476175
568.0648326608444
566.4546468994819
568.0658233036128
566.8265627535525
568.0666534180525
566.3700877336701
568.0737141952001
566.8698864970409
568.0738805472342
566.346062634925
568.0741058702838
566.7560668928091
568.0742736033554
566.5226881274762
568.0766116571173
566.7623039367166
568.0781058469998
566.4993337251583
568.0784560418078
566.7807286877303
568.0773101948362
566.4984248521001
568.0805485598529
566.5099885622435
568.0802474971645
566.776031489705
568.0809723829975
566.5003905685

566.7292353845551
568.326804673551
566.9751919950735
568.3268581185905
566.7755798744505
568.3273953572046
566.9865393088553
568.3274050571267
566.8142183269265
568.3344792399839
567.0293526676796
568.3314825165496
566.7716428772106
568.3310046631577
566.7689131980594
568.3342639691377
566.7686593037353
568.3339987914566
566.7690471486336
568.3332382569974
566.7703580445748
568.3390506433547
566.8379290855511
568.3402512682801
567.044354425028
568.3410353002926
566.7100091361372
568.3409782337251
567.0848923528389
568.3399716688634
566.7098661708987
568.3416332654817
566.7077091278492
568.342100984539
567.1177696537441
568.3501682931765
566.7294633068675
568.3509056170678
567.0822785255189
568.3462781210311
566.7291659594939
568.3516559120583
566.6996362942301
568.3514814760792
567.1074027229247
568.3530048507481
566.6846486866248
568.3540149875354
567.1318589098266
568.3542245993763
566.6696392457822
568.354569178705
567.1117124281881
568.3554659486988
566.7539905866211
568.3558320192

568.559908977083
567.0072984170675
568.5595293984748
567.2428128488448
568.5597524858496
567.0068947816678
568.5606539915358
567.0951649319136
568.5625954295641
567.2204923147908
568.5628975703451
566.9831193643023
568.561442762482
567.2632808888875
568.5625673297523
566.9828789318402
568.5628195805833
566.9828838932159
568.5618747014075
566.9851977567023
568.5659960664613
566.8560056981294
568.5676663253715
567.3107139946867
568.5670036560458
566.9363318686378
568.5728459491189
567.0083962261727
568.5730052243414
567.2711683484475
568.5733563144818
566.9463801303237
568.5737647084716
567.3124091556028
568.5737075860261
566.9458635189682
568.573984110902
566.9058711738361
568.5820262979103
567.2538750981016
568.574892682435
567.0459630960436
568.5828743867272
567.0459668925798
568.5842138301665
567.2753523624655
568.5845085378522
566.9909431818636
568.5855191226906
567.3147531675426
568.5856126678719
567.023166097532
568.5856803710112
567.2822568734806
568.5865652700252
567.01046644404

In [34]:
for t in ibm.sample: print(t.upd.beta.T)

[[ 0.88080206  0.12010337 -0.07747527]]
[[ 0.81438535 -1.16047985 -0.74931704]]
[[ 0.985417   -0.97041991 -0.76073637]]
[[ 0.94494615 -0.504805   -0.22644619]]
[[ 0.92907774 -0.26821758 -0.35627456]]
[[ 0.777323   -0.88530413 -0.38440876]]
[[ 0.93791438 -0.63874281 -0.52402629]]
[[ 0.90778475 -0.53812765 -0.32593765]]
[[ 0.8267352  -0.44704118 -0.13727789]]
[[ 1.01469764 -0.63353224 -0.93823508]]
[[ 1.04467142 -0.52413902 -0.67554349]]
[[ 0.93984858 -0.59515867 -0.42292756]]
[[0.91904337 0.16376095 0.07060386]]
[[ 0.74167999 -0.19997897  0.00473264]]
[[ 0.74820185 -0.38364641 -0.1567618 ]]
[[ 0.79485416 -0.44373951 -0.23929155]]
[[ 0.82323782 -0.59586638 -0.35332112]]
[[ 0.94755997 -0.50360156 -0.55946042]]
[[ 0.89187744 -0.60721542 -0.38189926]]
[[ 0.90708346 -0.52288823 -0.40013466]]
[[ 0.88261427 -0.53373106 -0.33905709]]
[[ 0.89038768 -0.49429124 -0.3505483 ]]
[[ 0.84497053 -0.40038505 -0.17479861]]
[[ 0.83029532 -0.47911354 -0.27647627]]
[[ 0.77315605 -0.15757249  0.14255413]]
[[ 

In [47]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math
from scipy.optimize import minimize

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.r = rnd((1,1))
        self.R = self.r.T@self.r
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['R','Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.R = self.r.T@self.r
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ inv(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
    def fun2min(self,column): # This is the function we will be minimizing.
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        self.run() # Run the Kalman filter with the new parameters.
        print (self.ll) # Report progress.
        return -self.ll # Return the negative of the lig likelihood.
    def F_constraint(self,column):
        self.pars.unpack(column) # Unpack the column into the parameter matrices.
        F = self.pars.F
        I = np.eye(F.shape[0])
        try: 
            an_inverse = inv(I-F)
            return 1
        except: return -1
    def estimate(self,tol=0.01, maxit = 500):
        response = 'Y'
        while response != 'N':
            starting_column = self.pars.pack()
            constr = {"type": "ineq", "fun": self.F_constraint}
            opt = {"maxiter": maxit}
            solution = minimize(
                fun = self.fun2min,
                x0 = starting_column,
                method = "COBYLA",
                constraints = constr,
                options = opt)
            self.optimum = solution
            self.pars.unpack(solution.x)
            self.run()
            print ('SUCCES: '+ str(solution.success) +': '+solution.message)
            response = input('Should I continue? (Y/N)')

        
        

In [48]:
ibm = the_model('IBM.csv')

In [49]:
ibm.estimate(tol = 0.001)

-520.8331524261055
-679.8729868966172
-593.9711938503076
-593.771366336283
-592.6989154903324
-606.1745236935153
-564.6854589786499
-596.6478284116059
-583.2216345585034
-603.0220860404955
-521.2346498852139
-521.074910386103
-521.0521163273628
-524.0464398171022
-524.824590240158
-522.82218750578
-520.8664242925316
-520.8904575330304
-520.8740682775924
-520.854211112397
-520.8695813580495
-520.8593896742937
-520.8450462263103
-520.8560491140831
-520.8486225475874
-521.5436646158486
-521.4484756140556
-521.0536412599479
-521.0697947859913
-521.3183871089396
-521.911156846996
-521.3560739185652
-521.0546741204037
-521.1025400894678
-800.7378841089823
142.03271986285702
-220.04785180618074
78.73844675482322
-208.2640636322837
168.99067190646755
140.09077590380411
161.11429009563918
137.7983094492605
149.8919046609653
210.78930264189563
-184.6154378735097
258.4005810069259
472.7149249118311
228.74134082934046
226.04630626944248
282.6808823964136
218.63118055111715
396.2606956107997
295.96

525.5342123957814
525.5414152076431
525.469154519086
525.5434764849849
525.5512979192225
525.553058373483
525.5052704274419
525.5525643613834
525.4356777600756
525.5233684756871
525.5037850442569
525.5408426957434
525.5746176506494
525.5590673819532
525.607526666786
525.6244894624858
525.5793826147537
525.6273027815954
525.7164306760332
525.7129654900185
525.714139758795
525.7164402451824
525.6959898597372
525.7144564421191
525.6644170780363
525.7196310528685
525.6041262078795
525.6893004085583
525.6872982931116
525.7165944949988
525.6057256089274
525.5883981702405
525.6779202639354
525.7117421933502
525.710525384249
525.7211586159492
525.760918210967
525.7617952546658
525.7724521643432
525.841548825388
525.8712617667228
525.882695645254
525.8856902668082
525.9007700691423
525.8548659677019
525.8949908768898
SUCCES: False: Maximum number of function evaluations has been exceeded.
Should I continue? (Y/N)Y
525.9007700691423
363.9688386788871
477.10075359249697
510.55889811588673
509.626

561.9457289530252
273.8362276349577
561.939369371057
273.8326930294489
562.0371236277593
268.8049279087088
562.1103869529161
439.1454919377834
562.129992956999
298.60105419657435
562.0777381741369
441.988959474031
561.8646867620403
302.4467117506508
562.0887000107896
302.40215956454466
562.1931812335833
324.2683278743901
562.1647385023614
443.41624622578894
562.1576506314564
324.2916137588352
562.6534498841784
302.7039672715131
562.6569214788123
441.9947722973703
562.871914770782
270.9362049271659
563.1239629341902
441.56731865888446
563.1556440214943
279.1494670390114
563.0362198799774
441.7560867981038
563.2004002389735
272.09776892593874
562.7691869271296
441.47562390282474
563.2288605464105
275.55432070453287
563.1973356318595
441.71020682089204
563.2097690778864
275.5829702789514
563.2596792013047
275.32899224784796
563.2496169618445
441.7019590960132
563.3171107632297
274.1306689413859
563.3571019871401
441.36900233719894
563.2650440408315
265.8394301236281
563.6339455802843
251.

568.2635949882618
568.714092084157
567.6784702219301
568.7172117074748
568.0483545574998
568.7179635164447
567.4341531984628
568.7151287823376
568.0899297838122
568.7144358506067
568.4029751870149
568.7197034794835
568.59790541074
568.6652814066583
568.4971708588569
568.7267818681796
568.3972104215188
568.7491614013567
563.6294624107322
568.7500119882541
568.2807542374558
568.7506461646248
567.7299415393558
568.7526981486392
568.1305450486116
568.7529379391763
568.4773222502179
568.766772235084
567.5299039929627
568.7681571258795
568.1638888489579
568.7507934015637
568.598541457579
568.7719589265001
568.6310115259424
568.772243653746
568.4217149210903
568.7841201084976
564.0831937444647
568.791109723918
568.3752703444649
568.8067228430214
567.7636585848117
568.8096016786262
568.1621154657535
568.8131264015286
568.503632716045
568.8115610770509
568.6460684720862
568.8130517971524
568.7690382907308
568.8143918424851
568.7959920407692
568.8136187872515
568.8092952393536
568.8118105815618


570.6403295688552
570.5457311081501
570.6326361326799
570.6015490244006
570.6415371589086
570.6180264186332
570.6421199963023
570.6155840443536
570.641355058903
570.6281400408393
570.6445069067045
570.6160602020677
570.6477484393437
570.6469925132084
570.6488560035684
570.641012266027
570.6484652870937
570.6122957239942
570.6500602298275
570.4614311294713
570.6435639556378
570.6375107580094
570.6417730422227
570.6579241206257
570.6559429693968
570.6591273899948
570.6603652596873
570.6654282326805
570.6583178538825
570.5337804073122
570.6267396307135
569.7575344993237
570.657067198178
570.1342976329083
570.6576121605556
570.4801186212276
570.6667890727595
570.6436039269098
570.6671097527499
570.6523440128828
570.6675938284767
570.6307879894499
570.6670785212759
570.657006704336
570.6637275366098
570.6514123888546
570.6721616067672
570.5282074522797
570.6725282394642
570.1969535731631
570.6734493439382
570.4740272148539
570.6737509216209
569.5636148187266
570.6740092568471
570.4743534309

In [50]:
for t in ibm.sample: print(t.upd.beta.T)

[[ 1.1871088  -0.81748919  0.11430427]]
[[ 1.20708753 -0.88154801 -1.90840139]]
[[ 1.12977093 -1.04382297 -0.68492571]]
[[ 0.99584152 -0.52484394 -0.42852629]]
[[ 0.90778844 -0.48702677 -0.58922085]]
[[ 0.8842864  -0.5825073  -0.47936008]]
[[ 0.94219514 -0.57561318 -0.60109659]]
[[ 0.90783711 -0.50375077 -0.29389702]]
[[ 0.83999927 -0.37050585 -0.23817797]]
[[ 0.96484285 -0.67301266 -1.11266316]]
[[ 1.00669452 -0.6720047  -0.54284753]]
[[ 0.94413896 -0.53657097 -0.48278501]]
[[ 0.78610231 -0.06719029  0.59514798]]
[[ 0.69864189 -0.10516126 -0.08413831]]
[[ 0.74822758 -0.29729846 -0.09308369]]
[[ 0.80886348 -0.35337757 -0.28155774]]
[[ 0.85050984 -0.45867269 -0.40015279]]
[[ 0.92650775 -0.55129921 -0.64584502]]
[[ 0.91007525 -0.52823521 -0.3225589 ]]
[[ 0.90083376 -0.4832476  -0.50934708]]
[[ 0.88624894 -0.48692436 -0.30846045]]
[[ 0.88827207 -0.46475038 -0.42344519]]
[[ 0.84963579 -0.39776143 -0.14493976]]
[[ 0.84825391 -0.40840379 -0.39413576]]
[[ 0.77648692 -0.23828272  0.23926361]]


<h3>Scratchwork:</h3>

In [2]:
import numpy as np
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [4]:
a = 1
b = 5

In [6]:
try: 
    c = a+b
    print (c)
except: 
    print("can't do it!")

6


In [7]:
b = str(5)

In [8]:
b

'5'

In [9]:
a + b

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [10]:
try: 
    c = a+b
    print (c)
except: 
    print("can't do it!")

can't do it!


In [37]:
ibm.optimum

     fun: -568.7416530655037
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 5000
  status: 2
 success: False
       x: array([[ 4.49576115e-01],
       [ 3.03207535e-01],
       [-1.84768233e-01],
       [-1.93162433e-01],
       [-8.09788467e-02],
       [ 8.94902168e-02],
       [ 1.11459876e-02],
       [ 5.51689570e-01],
       [ 2.93312232e-01],
       [ 1.05490681e+00],
       [-1.06144284e-03],
       [ 1.34035334e+00],
       [ 3.74754699e-01],
       [-6.25667989e-01],
       [ 3.09870988e-03],
       [-4.71780341e-01],
       [ 1.73981390e-01],
       [ 1.08587595e+00],
       [ 4.31495518e-01],
       [ 1.53683807e+00],
       [ 1.13443642e+00],
       [ 9.09768396e-01],
       [ 7.48043401e-01],
       [-1.58076825e-02],
       [ 2.05178270e-01],
       [-5.02723801e-02],
       [ 7.75065236e-02],
       [ 2.51147832e-01],
       [ 5.05367805e-01],
       [ 6.46334999e-01],
       [-6.43043810e-01],
       [ 6.46172942e-01],
  